# DATA Pre-processing

In [ ]:
import pandas as pd
merged_df = pd.read_csv('merged_df.csv')

We will perform the following preprocessing steps: 

1. Drop unnecessary columns
2. Convert Date Columns to datetime Format
3. Hnadle Missing Values
4. Remove Duplicate Rows
5. Handle Outliers using Winsorization
6. Scale Only Stock Market Features 

### 1. Drop unnecessary columns 

In [29]:
import numpy as np
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import MinMaxScaler

if 'Unnamed: 0' in merged_df.columns:
    merged_df.drop(columns=['Unnamed: 0'], inplace=True)
merged_df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Date,id,type,sectionName,publicationDate,webTitle,webUrl,headline,body
0,4745.200195,4754.330078,4722.669922,4742.830078,3743050000,0.0,0.0,2024-01-02,tv-and-radio/2024/jan/03/detroiters-sitcom-tim...,article,Television & radio,2024-01-02,"Before I Think You Should Leave, there was Det...",https://www.theguardian.com/tv-and-radio/2024/...,"Before I Think You Should Leave, there was Det...",<p>Pinning down why Tim Robinson is funny is v...
1,4745.200195,4754.330078,4722.669922,4742.830078,3743050000,0.0,0.0,2024-01-02,environment/2024/jan/02/climate-crisis-2023-wa...,article,Environment,2024-01-02,Climate crisis: 2023 was UK’s second-hottest y...,https://www.theguardian.com/environment/2024/j...,Climate crisis: 2023 was UK’s second-hottest y...,<p>The UK had its second-hottest year on recor...
2,4745.200195,4754.330078,4722.669922,4742.830078,3743050000,0.0,0.0,2024-01-02,business/2024/jan/02/uk-grocery-inflation-food...,article,Business,2024-01-02,UK shop inflation sticks at 4.3% despite lower...,https://www.theguardian.com/business/2024/jan/...,UK shop inflation sticks at 4.3% despite lower...,<p>Shop prices continued to rise at 4.3% in De...
3,4745.200195,4754.330078,4722.669922,4742.830078,3743050000,0.0,0.0,2024-01-02,stage/2024/jan/02/oliver-emanuel-obituary,article,Stage,2024-01-02,Oliver Emanuel obituary,https://www.theguardian.com/stage/2024/jan/02/...,Oliver Emanuel obituary,"<p>Oliver Emanuel, who has died aged 43 from b..."
4,4745.200195,4754.330078,4722.669922,4742.830078,3743050000,0.0,0.0,2024-01-02,business/live/2024/jan/02/aldi-lidl-enjoy-reco...,liveblog,Business,2024-01-02,Tesla overtaken by China’s BYD as world’s top-...,https://www.theguardian.com/business/live/2024...,Tesla overtaken by China’s BYD as world’s top-...,"<div id=""block-6594260e8f083d1e9a9aea1a"" class..."


### 2. Convert Date Columns to datetime 

In [31]:
# Ensuring proper date format
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df['publicationDate'] = pd.to_datetime(merged_df['publicationDate'])
# Show changes
print("\n After Converting Dates to datetime:")
print(merged_df.info())


 After Converting Dates to datetime:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1687 entries, 0 to 1686
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Open             1687 non-null   float64       
 1   High             1687 non-null   float64       
 2   Low              1687 non-null   float64       
 3   Close            1687 non-null   float64       
 4   Volume           1687 non-null   int64         
 5   Dividends        1687 non-null   float64       
 6   Stock Splits     1687 non-null   float64       
 7   Date             1687 non-null   datetime64[ns]
 8   id               1687 non-null   object        
 9   type             1687 non-null   object        
 10  sectionName      1687 non-null   object        
 11  publicationDate  1687 non-null   datetime64[ns]
 12  webTitle         1687 non-null   object        
 13  webUrl           1687 non-null   object        
 14  he

### 3. Handle Missing Values 

In [33]:
# Show missing values BEFORE handling them
print("\n Missing Values Count (Before Handling):")
print(merged_df.isnull().sum())


 Missing Values Count (Before Handling):
Open               0
High               0
Low                0
Close              0
Volume             0
Dividends          0
Stock Splits       0
Date               0
id                 0
type               0
sectionName        0
publicationDate    0
webTitle           0
webUrl             0
headline           0
body               0
dtype: int64


No missing values were found in any column. This means the dataset does not require imputation, and missing value handling is not needed in this case.

### 4. Remove duplicate rows

In [36]:
# Count duplicate rows BEFORE removal
num_duplicates = merged_df.duplicated().sum()
print(f"\n Number of Duplicate Rows (Before Removal): {num_duplicates}")

# Drop duplicate rows
merged_df.drop_duplicates(inplace=True)

# Show dataset shape AFTER removing duplicates
print(f"\n Number of Duplicate Rows (After Removal): {merged_df.duplicated().sum()}")
print(f" Dataset Shape After Removing Duplicates: {merged_df.shape}")


 Number of Duplicate Rows (Before Removal): 5

 Number of Duplicate Rows (After Removal): 0
 Dataset Shape After Removing Duplicates: (1682, 16)


### 5. Handle Outliers

In [38]:
# Show stock price summary BEFORE Winsorization
print("\n Stock Price Summary (Before Winsorization):")
print(merged_df[['Open', 'High', 'Low', 'Close']].describe())

# Apply Winsorization (Capping extreme values)
def winsorize_series(series, limits=[0.01, 0.01]):
    return winsorize(series, limits=limits)

for col in ['Open', 'High', 'Low', 'Close']:
    merged_df[col] = winsorize_series(merged_df[col], limits=[0.01, 0.01])

# Show stock price summary AFTER Winsorization
print("\n Stock Price Summary (After Winsorization):")
print(merged_df[['Open', 'High', 'Low', 'Close']].describe())


 Stock Price Summary (Before Winsorization):
              Open         High          Low        Close
count  1682.000000  1682.000000  1682.000000  1682.000000
mean   5422.253513  5447.243239  5397.229180  5423.704060
std     362.272588   363.299674   361.288070   362.140062
min    4690.569824  4721.490234  4682.109863  4688.680176
25%    5132.379883  5165.620117  5104.350098  5127.790039
50%    5433.669922  5478.310059  5406.959961  5434.430176
75%    5726.520020  5741.029785  5702.830078  5722.259766
max    6089.029785  6099.970215  6079.979980  6090.270020

 Stock Price Summary (After Winsorization):
              Open         High          Low        Close
count  1682.000000  1682.000000  1682.000000  1682.000000
mean   5422.457416  5447.322808  5397.333586  5423.993778
std     361.458971   362.911485   360.676038   361.414349
min    4739.129883  4744.229980  4714.819824  4739.209961
25%    5132.379883  5165.620117  5104.350098  5127.790039
50%    5433.669922  5478.310059  5406.9

Winsorization is used to cap extreme values in the stock price data, ensuring that outliers don’t skew the analysis.


### 6. Scale Only Stock Market Features

In [41]:
# Define stock market feature columns before using them
stock_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']

# Show numerical feature summary BEFORE scaling
print("\nStock Price Summary (Before Scaling):")
print(merged_df[stock_columns].describe())

# Apply MinMax Scaling (Normalize values between 0 and 1)
scaler = MinMaxScaler()
merged_df[stock_columns] = scaler.fit_transform(merged_df[stock_columns])

# Show numerical feature summary AFTER scaling
print("\n Stock Price Summary (After MinMax Scaling):")
print(merged_df[stock_columns].describe())


Stock Price Summary (Before Scaling):
              Open         High          Low        Close        Volume  \
count  1682.000000  1682.000000  1682.000000  1682.000000  1.682000e+03   
mean   5422.457416  5447.322808  5397.333586  5423.993778  3.954992e+09   
std     361.458971   362.911485   360.676038   361.414349  7.819356e+08   
min    4739.129883  4744.229980  4714.819824  4739.209961  1.757720e+09   
25%    5132.379883  5165.620117  5104.350098  5127.790039  3.530380e+09   
50%    5433.669922  5478.310059  5406.959961  5434.430176  3.835170e+09   
75%    5726.520020  5741.029785  5702.830078  5722.259766  4.140560e+09   
max    6069.390137  6089.839844  6061.060059  6084.189941  8.223220e+09   

       Dividends  Stock Splits  
count     1682.0        1682.0  
mean         0.0           0.0  
std          0.0           0.0  
min          0.0           0.0  
25%          0.0           0.0  
50%          0.0           0.0  
75%          0.0           0.0  
max          0.0     

- Before scaling:   
Open, High, Low, Close values range from 4739 to 6084  
Volume values are ranging from 1.75B to 8.22B
- After scaling  
  All values are between 0 and 1  
  The relative differences between values are preserved  
  Ensures equal contribution from all features